In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
# Cargo los dataframes
df_plataformas = pd.read_csv('plataformas.csv')
df_scores = pd.read_csv('scores.csv')

In [22]:
# Calculo el rating promedio de cada película y lo agrego al dataframe de películas
movie_ratings = df_scores.groupby("movieId")["score"].mean()
movies_df = df_plataformas.join(movie_ratings, on="id")

In [23]:
# Elimino duplicados
df_scores = df_scores.drop_duplicates(subset=["userId", "movieId"])

In [ ]:
# Creo una matriz de ratings, donde las filas son usuarios y las columnas son películas
ratings_matrix = df_scores.pivot(index="userId", columns="movieId", values="score")

In [ ]:
# Calculo la similitud entre los usuarios utilizando la distancia del coseno
user_similarity = cosine_similarity(ratings_matrix)

In [ ]:
# Creo una función para obtener las películas mejor valoradas por los usuarios similares
def get_similar_movies(userId, num_similar_users=5, num_recommendations=5):
    # Obtengo los usuarios más similares al usuario actual
    similar_users = pd.Series(user_similarity[userId]).sort_values(ascending=False)[1:num_similar_users+1].index
    # Obtengo las películas vistas por los usuarios similares que el usuario actual no haya visto aún
    similar_users_ratings = ratings_matrix.loc[similar_users]
    user_ratings = ratings_matrix.loc[userId]
    movies_to_recommend = similar_users_ratings[user_ratings.isnull()].sum().sort_values(ascending=False)[:num_recommendations]
    # Agrego información de las películas recomendadas
    recommendations_df = pd.DataFrame(movies_to_recommend, columns=["score"])
    recommendations_df = recommendations_df.join(movies_df.set_index("movieId"))
    return recommendations_df

# Ejemplo de recomendación para el usuario con id 1
get_similar_movies(1)